In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_grad=pd.read_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_insercion_laboral_titulados_universitarios_2019/EILU_GRADUADOS/CSV/EILU_GRAD_2019.csv',
                    sep='\t')

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_grad.shape

(31651, 299)

In [4]:
for col in df_grad.columns:
    print(col,df_grad[col].dtype)

IDENT int64
SEXO int64
EDAD int64
NACIO int64
NACIO1 object
NACIO2 object
PAIS_NACI int64
TITU int64
RAMA int64
AMBITO int64
T_UNIV int64
DISCA int64
PROXY int64
LUG_RES_PAIS int64
PAIS_NAC_PADRE int64
ESTUDIOS_PADRE int64
PAIS_NAC_MADRE int64
ESTUDIOS_MADRE int64
EST_B1 int64
EST_B2_1 object
EST_B2_2 object
EST_B2_3 object
EST_B2_4 object
EST_B2_5 object
EST_M1 int64
EST_M2 object
EST_M3 object
EST_M4 object
EST_MES int64
EST_B8 int64
SAT1 int64
SAT2 object
EST_B11_1 int64
EST_B11_2 int64
EST_B11_3 int64
EST_B11_4 int64
EST_B11_5 int64
EST_B11_6 int64
EST_B11_7 int64
EST_B12 object
EST_B13_AMB1 object
EST_B14_1 object
EST_B15_1 object
EST_B13_AMB2 object
EST_B14_2 object
EST_B15_2 object
EST_B13_AMB3 object
EST_B14_3 object
EST_B15_3 object
EST_B16 object
EST_B17_M1 object
EST_B18_1 object
EST_B19_1 object
EST_B17_M2 object
EST_B18_2 object
EST_B19_2 object
EST_B17_M3 object
EST_B18_3 object
EST_B19_3 object
EST_B21 object
EST_B22 object
EST_B23 object
EST_B24 object
EST_B25_FA1 objec

#### 'IDENT'
**Índice** único del entrevistado.

In [5]:
df_grad['IDENT'].unique()

array([    1,     2,     3, ..., 31649, 31650, 31651])

#### 'SEXO'
**Sexo**: '1'=Hombre, '2'='Mujer'

In [6]:
df_grad['SEXO'].unique() # No missing values

array([1, 2])

In [7]:
def proportions(col):
    print (df_grad[col].value_counts()/df_grad[col].count())
    return 

In [8]:
proportions('SEXO')

2    0.570472
1    0.429528
Name: SEXO, dtype: float64


#### 'EDAD'
**Edad** del entrevistado: 1=Menores de 30 años, 2=De 30 a 34 años, 3=De 35 y más años. 

In [9]:
df_grad['EDAD'].unique()

array([1, 2, 3])

In [10]:
proportions('EDAD')

1    0.496256
2    0.278822
3    0.224922
Name: EDAD, dtype: float64


#### 'NACIO'
**Nacionalidad** del entrevistado: 1=Española, 2=Española y otra, 3=Otra nacionalidad

In [11]:
df_grad['NACIO'].unique()

array([1, 2, 3])

In [12]:
proportions('NACIO')

1    0.978137
3    0.011027
2    0.010837
Name: NACIO, dtype: float64


#### 'NACIO1'
**País de primera nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.

In [13]:
df_grad['NACIO1'].unique()

array([' ', '2', '9', '1'], dtype=object)

In [14]:
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])

,NACIO1
,unique
NACIO,
1,[ ]
2,"[2, 1, 9]"
3,"[9, 2, 1]"


In [15]:
# Se imputa nacionalidad de un país de la UE (1) a quien declaró solo nacionalidad española en variable 'NACIO'
df_grad['NACIO1'].mask(df_grad['NACIO']==1,'1',inplace=True)
df_grad['NACIO1']=df_grad['NACIO1'].astype('int')
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])

,NACIO1
,unique
NACIO,
1,[1]
2,"[2, 1, 9]"
3,"[9, 2, 1]"


In [16]:
proportions('NACIO1')

1    0.985751
2    0.012511
9    0.001738
Name: NACIO1, dtype: float64


#### 'NACIO2'
**País de segunda nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.

In [17]:
df_grad['NACIO2'].unique()

array([' ', '1', '2'], dtype=object)

In [18]:
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])
# Quienes declaran 'nacionalidad española y otra' (2 en 'NACIO') no responden sobre 'NACIO2'

,NACIO2
,unique
NACIO,
1,[ ]
2,[ ]
3,"[ , 1, 2]"


In [19]:
# Se imputa 'NACIO2'=1 (UE)  a quienes declararon 'nacionalidad española y otra' (2 en 'NACIO') y la primera ...
# ... no es nacionalidad EU ('NACIO1' distinta de 1), pues su segunda nacionalidad es española.
df_grad['NACIO2'].mask((df_grad['NACIO']==2)&(df_grad['NACIO1']!=1),1,inplace=True)
# Se imputa None al resto de no respuestas:
df_grad['NACIO2'].replace({' ':None},inplace=True)
df_grad['NACIO2']=df_grad['NACIO2'].astype('float')
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])

,NACIO2
,unique
NACIO,
1,[nan]
2,"[1.0, nan]"
3,"[nan, 1.0, 2.0]"


In [20]:
proportions('NACIO2')

1.0    0.970588
2.0    0.029412
Name: NACIO2, dtype: float64


#### 'PAIS_NACI'
**País de nacimiento**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC

In [21]:
df_grad['PAIS_NACI'].unique()

array([1, 3, 2])

#### 'TITU'
**Titulación**:  
011101=Pedagogía, 011201=Educación infantil, 011301=Educación primaria, 011401=Otros maestros, 011901=Educación social,   
021101=Audiovisual, imagen y multimedia, 021201=Diseño, 021301=Bellas artes, 021302=Historia del arte, 021401=Conservación y restauración, 021502=Música y Artes escénicas, 022201=Arqueología, 022202=Historia, 022301=Filosofía, 022901=Humanidades, 023101=Lengua inglesa, 023102=Lenguas clásicas, 023103=Otras lenguas extranjeras, 023104=Traducción e interpretación, 023201=Lenguas y dialectos españoles, 023202=Literatura, 023901=Lenguas modernas y aplicadas,   
031101=Economía, 031201=Política y gestión pública, 031202=Relaciones internacionales, 031301=Psicología, 031401=Antropología social y cultural y Estudios y gestión de la cultura, 031402=Criminología, 031404=Geografía, 031406=Sociología e Igualdad de género, 032101=Comunicación, 032102=Periodismo, 032201=Información y documentación,   
041201=Financiera y actuarial, 041202=Finanzas y contabilidad, 041301=Administración y empresa, 041302=Ciencias del trabajo, 041303=Gestión y administración pública, 041401=Marketing, 041402=Protocolo y eventos, 041403=Publicidad y relaciones públicas, 041601=Comercio, 042101=Derecho,   
051101=Biología, 051201=Bioquímica, 051202=Biotecnología, 051901=Biomedicina, 052101=Ciencias ambientales, 053101=Química, 053201=Ciencias del mar, 053202=Geografía y ordenación del territorio, 053203=Geología, 053301=Física, 054101=Matemáticas, 054201=Estadística,   
061301=Desarrollo de software y de aplicaciones e Ingeniería multimedia, 061901=Informática,   
071101=Ingeniería química industrial e Ingeniería medioambiental, 071301=Ingeniería de la energía, 071302=Ingeniería eléctrica, 071401=Ingeniería de computadores, 071402=Ingeniería de sonido e imagen, 071403=Ingeniería de telecomunicación, 071404=Ingeniería electrónica industrial y automática, 071405=Ingeniería en electrónica, 071501=Ingeniería en diseño industrial y desarrollo del producto, 071502=Ingeniería en tecnologías industriales, 071503=Ingeniería mecánica, 071601=Ingeniería aeronáutica, 071603=Ingeniería naval y oceánica, 071901=Ingeniería de organización industrial y Nanotecnología, 072101=Ciencia y tecnología de los alimentos e Ingeniería alimentaria, 072102=Enología, 072201=Ingeniería de materiales e Ingeniería textil, 072401	=Ingeniería de minas y energía, 073101=Arquitectura y Urbanismo y paisajismo, 073102=Ingeniería geomática, topografía y cartografía, 073201=Arquitectura técnica, 073202=Ingeniería civil,   
081102=Ingeniería agraria y agroalimentaria, 081103=Ingeniería agrícola, agropecuaria y medio rural, 081201=Ingeniería horticultura y jardinería, 082101=Ingeniería forestal y montes, 084101=Veterinaria,   
091101=Odontología, 091201=Medicina, 091301=Enfermería, 091401=Ingeniería biomédica y de la salud, 091402=Óptica y optometría, 091501=Fisioterapia, 091502=Logopedia, 091503=Nutrición humana y dietética, 091504=Podología, 091505=Terapia ocupacional, 091601=Farmacia, 092301=Trabajo social,   
101401=Actividad física y del deporte, 101501=Turismo, 104101=Náutica y transporte marítimo, 104103=Servicio de transporte terrestre y Servicio de transporte aéreo, 109999=Servicios (otros estudios).

In [22]:
df_grad['TITU'].unique()

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999])

#### 'RAMA'
**Rama de conocimiento** de la titulación:  
1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud

In [23]:
df_grad['RAMA'].unique()

array([3, 1, 4, 5, 2])

In [24]:
proportions('RAMA')

3    0.458248
4    0.211842
5    0.141796
1    0.100376
2    0.087738
Name: RAMA, dtype: float64


In [25]:
# Varias titulaciones adscritas a más de una rama del conocimiento
df_grad[['RAMA','TITU']].groupby('TITU').agg(['unique'])

,RAMA
,unique
TITU,
11101,"[3, 1]"
11201,[3]
11301,[3]
11401,[3]
11901,[3]
...,...
101401,"[3, 5]"
101501,[3]


In [26]:
df_grad[['RAMA','TITU']].groupby('RAMA').agg(['unique'])

,TITU
,unique
RAMA,
1,"[11101, 21101, 21201, 21301, 21302, 21401, 21502, 22201, 22202, 22301, 22901, 23101, 23102, 23103, 23104, 23201, 23202, 23901, 31401, 31404, 32101, 32102, 32201, 53202, 73101]"
2,"[51101, 51201, 51202, 51901, 52101, 53101, 53201, 53203, 53301, 54101, 54201, 71901, 72101, 72102, 91402, 91503]"
3,"[11101, 11201, 11301, 11401, 11901, 21101, 23104, 31101, 31201, 31202, 31301, 31401, 31402, 31404, 31406, 32101, 32102, 32201, 41201, 41202, 41301, 41302, 41303, 41401, 41402, 41403, 41601, 42101, 53202, 54201, 91501, 92301, 101401, 101501, 104103, 109999]"
4,"[21101, 21201, 53203, 54101, 61301, 61901, 71101, 71301, 71302, 71401, 71402, 71403, 71404, 71405, 71501, 71502, 71503, 71601, 71603, 71901, 72101, 72201, 72401, 73101, 73102, 73201, 73202, 81102, 81103, 81201, 82101, 91401, 104101, 104103, 109999]"
5,"[31301, 51101, 51901, 72101, 84101, 91101, 91201, 91301, 91402, 91501, 91502, 91503, 91504, 91505, 91601, 101401]"


#### 'AMBITO'
**Ámbito de estudio**: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.

In [27]:
df_grad['AMBITO'].unique()

array([  11,  112,  113,  211,   21,   22,   23,  311,   31,  313,   32,
         41,  413,   42,   51,   52,   53,   54,   61,   71,   72,   73,
         81,   82,   84,   91,  912,  913,   92, 1014, 1015,  109])

In [28]:
df_grad['AMBITO'].dtype

dtype('int64')

In [29]:
# Todas las titulaciones adscritas a un único ámbito de estudio.  
for row in range(0,len(df_grad[['TITU','AMBITO']].groupby('TITU'))):
    print (df_grad[['TITU','AMBITO']].groupby('TITU').agg(['unique']).iloc[row])

AMBITO  unique    [11]
Name: 11101, dtype: object
AMBITO  unique    [112]
Name: 11201, dtype: object
AMBITO  unique    [113]
Name: 11301, dtype: object
AMBITO  unique    [11]
Name: 11401, dtype: object
AMBITO  unique    [11]
Name: 11901, dtype: object
AMBITO  unique    [211]
Name: 21101, dtype: object
AMBITO  unique    [21]
Name: 21201, dtype: object
AMBITO  unique    [21]
Name: 21301, dtype: object
AMBITO  unique    [21]
Name: 21302, dtype: object
AMBITO  unique    [21]
Name: 21401, dtype: object
AMBITO  unique    [21]
Name: 21502, dtype: object
AMBITO  unique    [22]
Name: 22201, dtype: object
AMBITO  unique    [22]
Name: 22202, dtype: object
AMBITO  unique    [22]
Name: 22301, dtype: object
AMBITO  unique    [22]
Name: 22901, dtype: object
AMBITO  unique    [23]
Name: 23101, dtype: object
AMBITO  unique    [23]
Name: 23102, dtype: object
AMBITO  unique    [23]
Name: 23103, dtype: object
AMBITO  unique    [23]
Name: 23104, dtype: object
AMBITO  unique    [23]
Name: 23201, dtype: obje

In [30]:
# 'AMBITO' determinado por 2, 3 o 4 primeros dígitos de 'TITU' excepto los siguientes casos: 
cond1=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:2])
cond2=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:3])
cond3=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:4])
df_grad[['AMBITO','TITU','RAMA']][cond1&cond2&cond3].value_counts()

AMBITO  TITU    RAMA
109     104101  4       109
        104103  3        47
                4        10
dtype: int64

In [31]:
df_grad[['AMBITO','TITU','RAMA']][cond1&cond2&cond3].shape

(166, 3)

In [32]:
# Varias ramas adscritas a más de un ámbito de estudio
df_grad[['RAMA','AMBITO']].groupby('AMBITO').agg(['unique'])

,RAMA
,unique
AMBITO,
11,"[3, 1]"
21,"[1, 4]"
22,[1]
23,"[1, 3]"
31,"[3, 1]"
32,"[3, 1]"
41,[3]
42,[3]


#### 'T_UNIV'
**Tipo de universidad**: 1=Universidad Pública presencial, 2=Universidad Pública a distancia, 3=Universidad Privada presencial, 4=Universidad Privada a distancia.

In [33]:
df_grad['T_UNIV'].unique()

array([2, 1, 4, 3])

In [34]:
proportions('T_UNIV')

1    0.818236
3    0.119838
2    0.034122
4    0.027803
Name: T_UNIV, dtype: float64


#### 'DISCA'
Tiene **discapacidad reconocida** superior al 33 %: 1=Sí, 2=No. 

In [35]:
df_grad['DISCA'].unique()

array([2, 1])

In [36]:
proportions('DISCA')

2    0.987804
1    0.012196
Name: DISCA, dtype: float64


In [37]:
# Discapacidad según tipo de universidad (en términos absolutos)
# Similar a value_counts(). Ventaja: permite cálculo de proporciones de acuerdo con dos criterios (a continuación)
df_grad[['T_UNIV','DISCA']].groupby(['T_UNIV','DISCA']).agg({'DISCA':'count'}) 

DISCA
T_UNIV DISCA       
1      1        315
       2      25583
2      1         29
       2       1051
3      1         23
       2       3770
4      1         19
       2        861

In [38]:
def dist_condicionada(col,cond,rev=False):
    if rev==False: 
        out=df_grad[[cond,col]].groupby([cond,col]).agg({col:'count'}
                ).div(df_grad[[cond,col]].groupby(cond).agg({col:'count'}),level=cond)
    else: 
        out=df_grad[[col,cond]].groupby([col,cond]).agg({cond:'count'}
                ).div(df_grad[[col,cond]].groupby(col).agg({cond:'count'}),level=col)
    return out

In [39]:
# Proporción de discapacidad según el tipo de universidad: 
dist_condicionada('DISCA','T_UNIV')
# La proporción de estudiantes con discapacidad en las universidades a distancia públicas y privadas ...
# ... es al menos el doble que en las presenciales públicas y privadas respectivamente

DISCA
T_UNIV DISCA          
1      1      0.012163
       2      0.987837
2      1      0.026852
       2      0.973148
3      1      0.006064
       2      0.993936
4      1      0.021591
       2      0.978409

In [40]:
# Distribución de los ciclos formativos y enseñanzas deportivas de grado superior según motivacion para estudiar: 
dist_condicionada('EST_B11_5','EST_B8')
# De nuevo, sin diferencias notables. 

EST_B11_5
EST_B8 EST_B11_5           
1      1           0.124910
       2           0.875090
2      1           0.132417
       2           0.867583
3      1           0.150018
       2           0.849982
9      1           0.104348
       2           0.895652

In [41]:
# Proporción de cada tipo de universidad según discapacidad: 
dist_condicionada('DISCA','T_UNIV',rev=True)
# Los estudiantes con y sin discapacidad eligen universidad pública presencial mayoritariamente y ... 
# ... aproximadamente con la misma frecuencia. 
# Pero los estudiantes con discapacidad eligen la universidad a distancia (pública o privada) con más del doble ...
# ... de frecuencia que los estudiantes sin discapacidad. La relación se invierte para la privada presencial.

T_UNIV
DISCA T_UNIV          
1     1       0.816062
      2       0.075130
      3       0.059585
      4       0.049223
2     1       0.818263
      2       0.033616
      3       0.120582
      4       0.027539

#### 'PROXY'
**Tipo de entrevista**: 1=Entrevista directa con la persona seleccionada, 2=Entrevista proxy.

In [42]:
df_grad['PROXY'].unique()

array([1, 2])

In [43]:
proportions('PROXY')

1    0.987647
2    0.012353
Name: PROXY, dtype: float64


#### 'LUG_RES_PAIS'
**País de residencia actual**: 01=España,02=Reino Unido, 03=Alemania, 04=Francia, 05=Otro país de la Unión Europea, 06=Otro país europeo (fuera de la UE), 07=Un país de América del Norte, 08=Un país de América Central o América del Sur, 09=Un país de Asia, 10=Un país de África, 11=Un país de Oceanía, 99=NS/NC.

In [44]:
df_grad['LUG_RES_PAIS'].sort_values().unique() # No missing values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [45]:
proportions('LUG_RES_PAIS')

1     0.944109
2     0.015987
5     0.013586
3     0.006793
7     0.004644
4     0.004486
6     0.003791
8     0.002591
9     0.002117
11    0.001517
10    0.000379
Name: LUG_RES_PAIS, dtype: float64


#### 'PAIS_NAC_PADRE'
**País de nacimiento del padre**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC.

In [46]:
df_grad['PAIS_NAC_PADRE'].unique()

array([1, 9, 2, 3])

In [47]:
proportions('PAIS_NAC_PADRE')

1    0.954567
3    0.026855
2    0.011848
9    0.006730
Name: PAIS_NAC_PADRE, dtype: float64


#### 'ESTUDIOS_PADRE'
**Nivel de formación máximo alcanzado por el padre**: 1=No sabe leer o escribir, 2=Educación primaria incompleta (ha asistido menos de 5 años a la escuela), 3=Educación primaria completa, 4=Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental), 5=Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), 6=Enseñanzas profesionales de grado medio o equivalentes, 7=Enseñanzas profesionales de grado superior o equivalentes, 8=Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, 9=NS/NC.

In [48]:
df_grad['ESTUDIOS_PADRE'].unique()

array([2, 3, 7, 8, 5, 4, 6, 9, 1])

In [49]:
proportions('ESTUDIOS_PADRE')

8    0.285109
3    0.166219
4    0.159458
5    0.114025
6    0.076111
7    0.073078
9    0.069508
2    0.052921
1    0.003570
Name: ESTUDIOS_PADRE, dtype: float64


#### 'PAIS_NAC_MADRE'
**País de nacimiento de la madre**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC.

In [50]:
df_grad['PAIS_NAC_MADRE'].unique()

array([1, 3, 9, 2])

In [51]:
proportions('PAIS_NAC_MADRE')

1    0.947490
3    0.025908
2    0.014249
9    0.012353
Name: PAIS_NAC_MADRE, dtype: float64


#### 'ESTUDIOS_MADRE'
**Nivel de formación máximo alcanzado por la madre**: 1=No sabe leer o escribir, 2=Educación primaria incompleta (ha asistido menos de 5 años a la escuela), 3=Educación primaria completa, 4=Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental), 5=Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), 6=Enseñanzas profesionales de grado medio o equivalentes, 7=Enseñanzas profesionales de grado superior o equivalentes, 8=Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, 9=NS/NC.

In [52]:
df_grad['ESTUDIOS_MADRE'].unique()

array([3, 7, 2, 4, 5, 6, 8, 9, 1])

In [53]:
proportions('ESTUDIOS_MADRE')

8    0.272977
4    0.183849
3    0.167420
5    0.123440
6    0.091498
7    0.059177
2    0.051341
9    0.046349
1    0.003949
Name: ESTUDIOS_MADRE, dtype: float64


#### 'EST_B1'
¿Disfrutó de **alguna beca** durante sus estudios?: 1=Sí, 2=No, 9=NS/NC.

In [54]:
df_grad['EST_B1'].unique()

array([2, 1, 9])

In [55]:
proportions('EST_B1')

2    0.532874
1    0.466304
9    0.000821
Name: EST_B1, dtype: float64


#### 'EST_B2_1'
(Solo para respuesta Sí (1) en variable 'EST_B1')  
¿Disfrutó de una **beca general para el estudio**?: 1=Sí, 2=No, 9=NS/NC.

In [56]:
df_grad['EST_B2_1'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [57]:
def re_EST_B1(col):
    print (df_grad[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    # Se imputa respuesta no (2) a quienes manifestaron no haber disfrutado de beca (2 en 'EST_B1')
    df_grad[col].mask(df_grad['EST_B1']==2,'2',inplace=True)
    # Se imputa None al resto de no respuestas:
    df_grad[col].replace({' ':None},inplace=True)
    print (df_grad[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    return

In [58]:
re_EST_B1('EST_B2_1')

         EST_B2_1
           unique
EST_B1           
1       [1, 2, 9]
2             [ ]
9             [ ]
         EST_B2_1
           unique
EST_B1           
1       [1, 2, 9]
2             [2]
9          [None]


In [59]:
proportions('EST_B2_1')

2    0.61366
1    0.38185
9    0.00449
Name: EST_B2_1, dtype: float64


#### 'EST_B2_2'
(Solo para respuesta Sí (1) en variable 'EST_B1')  
¿Disfrutó de algún **premio o beca de excelencia**?: 1=Sí, 2=No, 9=NS/NC.

In [60]:
df_grad['EST_B2_2'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [61]:
re_EST_B1('EST_B2_2')

         EST_B2_2
           unique
EST_B1           
1       [2, 1, 9]
2             [ ]
9             [ ]
         EST_B2_2
           unique
EST_B1           
1       [2, 1, 9]
2             [2]
9          [None]


In [62]:
proportions('EST_B2_2')

2    0.951336
1    0.044174
9    0.004490
Name: EST_B2_2, dtype: float64


#### 'EST_B2_3'
(Solo para respuesta Sí (1) en variable 'EST_B1')  
¿Disfrutó de alguna **beca de colaboración** en la universidad?: 1=Sí, 2=No, 9=NS/NC.

In [63]:
df_grad['EST_B2_3'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [64]:
re_EST_B1('EST_B2_3')

         EST_B2_3
           unique
EST_B1           
1       [2, 1, 9]
2             [ ]
9             [ ]
         EST_B2_3
           unique
EST_B1           
1       [2, 1, 9]
2             [2]
9          [None]


In [65]:
proportions('EST_B2_3')

2    0.96019
1    0.03532
9    0.00449
Name: EST_B2_3, dtype: float64


#### 'EST_B2_4'
(Solo para respuesta Sí (1) en variable 'EST_B1')  
¿Disfrutó de alguna **beca de prácticas externas** o para prestar un servicio o desempeñar un trabajo fuera de la Universidad?: 1=Sí, 2=No, 9=NS/NC.

In [66]:
df_grad['EST_B2_4'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [67]:
re_EST_B1('EST_B2_4')

         EST_B2_4
           unique
EST_B1           
1       [2, 1, 9]
2             [ ]
9             [ ]
         EST_B2_4
           unique
EST_B1           
1       [2, 1, 9]
2             [2]
9          [None]


In [68]:
proportions('EST_B2_4')

2    0.957787
1    0.037723
9    0.004490
Name: EST_B2_4, dtype: float64


#### 'EST_B2_5'
(Solo para respuesta Sí (1) en variable 'EST_B1')  
¿Disfrutó de alguna **beca para realizar estudios fuera de España** (incluidas becas complemento a programas Erasmus)?: 1=Sí, 2=No, 9=NS/NC.

In [69]:
df_grad['EST_B2_5'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [70]:
re_EST_B1('EST_B2_5')

         EST_B2_5
           unique
EST_B1           
1       [2, 1, 9]
2             [ ]
9             [ ]
         EST_B2_5
           unique
EST_B1           
1       [2, 1, 9]
2             [2]
9          [None]


In [71]:
proportions('EST_B2_5')

2    0.889771
1    0.105739
9    0.004490
Name: EST_B2_5, dtype: float64


#### 'EST_M1'
(Solo para respuesta distinta de Sí (1) en variable 'EST_B2_5')  
¿Realizó parte de sus **estudios en el extranjero**?: 1=Sí, 2=No, 9=NS/NC.

In [72]:
df_grad['EST_M1'].unique()

array([2, 1, 9])

In [73]:
df_grad[['EST_B2_5','EST_M1']].groupby('EST_B2_5').agg(['unique'])
# Ya imputada respuesta Si (1) a quienes manifestaron haber disfrutado de beca para estudiar fuera de España

,EST_M1
,unique
EST_B2_5,
1,[1]
2,"[2, 1, 9]"
9,"[2, 1]"


In [74]:
proportions('EST_M1')

2    0.832201
1    0.167167
9    0.000632
Name: EST_M1, dtype: float64


#### 'EST_M2'
(Solo para respuesta Sí (1) en variable 'EST_B2_5')  
**Programa o beca de movilidad**: 1=Programa Erasmus, 2=Otros programas o becas dentro de la UE, 3=Otros programas o becas fuera de la UE, 9=NS/NC.  
Se añade respuesta 0=Ninguna. 

In [75]:
df_grad['EST_M2'].unique()

array([' ', '3', '1', '2', '9'], dtype=object)

In [76]:
df_grad[['EST_B2_5','EST_M2']].groupby('EST_B2_5').agg(['unique'])

,EST_M2
,unique
EST_B2_5,
1,"[3, 1, 2, 9]"
2,[ ]
9,[ ]


In [77]:
# Se imputa valor 0 ('Ninguna') a quienes manifestaron no haber disfrutado de beca para estudiar fuera ...
# ... de España (respuesta 2 en 'EST_B2_5').
df_grad['EST_M2'].mask(df_grad['EST_B2_5']=='2','0',inplace=True)
# Se imputa None al resto de no respuestas: 
df_grad['EST_M2'].replace({' ':None},inplace=True)
df_grad[['EST_B2_5','EST_M2']].groupby('EST_B2_5').agg(['unique'])

,EST_M2
,unique
EST_B2_5,
1,"[3, 1, 2, 9]"
2,[0]
9,[None]


In [78]:
proportions('EST_M2')

0    0.893784
1    0.092177
3    0.008640
2    0.005368
9    0.000032
Name: EST_M2, dtype: float64


In [79]:
# Distribución de tipos de becas de estudios en el extranjero entre quienes manifestaron haber disfrutado alguna: 
df_grad['EST_M2'][df_grad['EST_B2_5']=='1'].value_counts()/df_grad['EST_M2'][df_grad['EST_B2_5']=='1'].count()

1    0.867823
3    0.081340
2    0.050538
9    0.000299
Name: EST_M2, dtype: float64

#### 'EST_M3'
(Solo para respuesta Sí (1) en variable 'EST_M1')  
**País de destino** cuando estudió fuera de España: 01=Reino Unido, 02=Alemania, 03=Francia, 04=Italia, 05=Portugal, 06=Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia), 07=Otro país de la Unión Europea, 08=Otro país europeo, 11=Un país de América del Norte, 12=Un país de América Central o América del Sur, 21=Un país de Asia, 31=Un país de África, 41=Un país de Oceanía, 99=NS/NC.

In [80]:
df_grad['EST_M3'].unique()

array(['  ', '12', '03', '08', '05', '04', '02', '11', '06', '01', '07',
       '21', '99', '31', '41'], dtype=object)

In [81]:
df_grad[['EST_M1','EST_M3']].groupby('EST_M1').agg(['unique'])

,EST_M3
,unique
EST_M1,
1,"[12, 03, 08, 05, 04, 02, 11, 06, 01, 07, 21, 99, 31, 41]"
2,[ ]
9,[ ]


In [82]:
# Se imputa None a quienes no estudiaron fuera de España: 
df_grad['EST_M3'].replace({'  ':None},inplace=True)
df_grad[['EST_M1','EST_M3']].groupby('EST_M1').agg(['unique'])

,EST_M3
,unique
EST_M1,
1,"[12, 03, 08, 05, 04, 02, 11, 06, 01, 07, 21, 99, 31, 41]"
2,[None]
9,[None]


#### 'EST_M4'
(Solo para respuesta Sí (1) en variable 'EST_M1')  
**Duración** en meses del periodo en el que estudió fuera de España: 1=De 1 a 3 meses, 2=De 4 a 6 meses, 3=De 7 a 9 meses, 4=De 10 a 12 meses, 5=Más de un año, 9=NS/NC.  
Se añade respuesta 0=No estudió en el extranjero.

In [83]:
df_grad['EST_M4'].unique()

array([' ', '1', '4', '3', '2', '5', '9'], dtype=object)

In [84]:
df_grad[['EST_M1','EST_M4']].groupby('EST_M1').agg(['unique'])

,EST_M4
,unique
EST_M1,
1,"[1, 4, 3, 2, 5, 9]"
2,[ ]
9,[ ]


In [85]:
# Se imputa 0 a quienes no estudiaron fuera de España: 
df_grad['EST_M4'].mask(df_grad['EST_M1']==2,'0',inplace=True)
# Se imputa None al esto de no respuestas
df_grad['EST_M4'].replace({'  ':None},inplace=True)
df_grad[['EST_M1','EST_M4']].groupby('EST_M1').agg(['unique'])

,EST_M4
,unique
EST_M1,
1,"[1, 4, 3, 2, 5, 9]"
2,[0]
9,[ ]


#### 'EST_MES' 
¿Realizó parte de sus estudios para obtener su título de grado en **otra universidad española**? 1=Sí, 2=No, 9=NS/NC.

In [86]:
df_grad['EST_MES'].unique()

array([2, 1, 9])

In [87]:
proportions('EST_MES')

2    0.902626
1    0.096679
9    0.000695
Name: EST_MES, dtype: float64


#### 'EST_B8' 
**Motivo principal** por el que ha realizado sus **estudios** de grado: 1=Formarse pensando en su futuro laboral, 2=Ampliar conocimientos por satisfacción personal, 3=Otros motivos, 9=NS/NC.

In [88]:
df_grad['EST_B8'].unique()

array([1, 2, 3, 9])

In [89]:
proportions('EST_B8')

1    0.747938
2    0.160816
3    0.087612
9    0.003633
Name: EST_B8, dtype: float64


In [90]:
# Motivación según rama de conocimiento de la titulación (1=Artes y humanidades, 2=Ciencias,...
# ... 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud)
df_grad[['RAMA','EST_B8']].groupby(['RAMA','EST_B8']).agg({'EST_B8':'count'})

EST_B8
RAMA EST_B8        
1    1         1742
     2         1151
     3          274
     9           10
2    1         1892
     2          668
     3          201
     9           16
3    1        11443
     2         1844
     3         1176
     9           41
4    1         5325
     2          745
     3          595
     9           40
5    1         3271
     2          682
     3          527
     9            8

In [91]:
# Distribución de la motivación según rama de conocimiento de la titulación (1=Artes y humanidades, 2=Ciencias,...
# ... 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud):
dist_condicionada('EST_B8','RAMA')
# La motivación principal son las expectativas laborales para todas las ramas de conocimiento.
# Artes y humanidades es la rama que menos se escoge pensando en las expectativas laborales, seguida por Ciencias.
# También son estas, en ese orden, las que más se escogen para ampliar conocimientos por satisfacción personal. 

EST_B8
RAMA EST_B8          
1    1       0.548316
     2       0.362291
     3       0.086245
     9       0.003148
2    1       0.681311
     2       0.240547
     3       0.072380
     9       0.005762
3    1       0.788955
     2       0.127137
     3       0.081081
     9       0.002827
4    1       0.794183
     2       0.111111
     3       0.088740
     9       0.005966
5    1       0.728832
     2       0.151961
     3       0.117424
     9       0.001783

#### 'SAT1' 
¿Si tuviera que empezar de nuevo volvería a cursar estudios universitarios?  1=Sí, 2=No, 9=NS/NC.

In [92]:
df_grad['SAT1'].unique()

array([1, 2, 9])

In [93]:
proportions('SAT1')

1    0.857919
2    0.126410
9    0.015671
Name: SAT1, dtype: float64


In [94]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('SAT1','RAMA')
# La disposición a estudiar de nuevo es similar para todas las ramas excepto para los graduados ...
# ... en ciencias de la salud, más dispuestos a estudiar de nuevo que el resto. 

SAT1
RAMA SAT1          
1    1     0.843878
     2     0.142587
     9     0.013535
2    1     0.840115
     2     0.148001
     9     0.011883
3    1     0.849697
     2     0.133067
     9     0.017237
4    1     0.852349
     2     0.133482
     9     0.014169
5    1     0.913770
     2     0.069519
     9     0.016711

In [95]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según el sexo: 
dist_condicionada('SAT1','SEXO')
# Las diferencias entre sexos son pequeñas. 

SAT1
SEXO SAT1          
1    1     0.854579
     2     0.132402
     9     0.013019
2    1     0.860434
     2     0.121899
     9     0.017667

In [96]:
# Distribución de la disposición a estudiar según las motivaciones para estudiar un grado: 
dist_condicionada('SAT1','EST_B8')
# La disposición a estudiar de nuevo es ligeramente superior entre quienes estudiaron ... 
# ... principalmente para ampliar conocimientos por satisfacción personal. 

SAT1
EST_B8 SAT1          
1      1     0.853293
       2     0.131289
       9     0.015418
2      1     0.879175
       2     0.107859
       9     0.012967
3      1     0.860801
       2     0.120447
       9     0.018752
9      1     0.800000
       2     0.086957
       9     0.113043

#### 'SAT2'   
(Solo para respuesta Sí (1) en 'SAT1'?  
¿Si tuviera que empezar de nuevo volvería a estudiar la misma titulación?  1=Sí, 2=No, 9=NS/NC.

In [97]:
df_grad['SAT2'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [98]:
proportions('SAT2')

1    0.615115
2    0.239203
     0.142081
9    0.003602
Name: SAT2, dtype: float64


In [99]:
df_grad[['SAT1','SAT2']].groupby('SAT1').agg(['unique'])

,SAT2
,unique
SAT1,
1,"[2, 1, 9]"
2,[ ]
9,[ ]


In [100]:
# Se imputa respuesta No (2) a quienes declararon que no volverían a cursar estudios universitarios: 
df_grad['SAT2'].mask(df_grad['SAT1']==2,'2',inplace=True)
# Se imputa NS/NC (9) a quienes dieron esa respuesta al preguntarles si cursarían estudios universitarios de nuevo:
df_grad['SAT2'].replace({' ':'9'},inplace=True)
df_grad[['SAT1','SAT2']].groupby('SAT1').agg(['unique'])

,SAT2
,unique
SAT1,
1,"[2, 1, 9]"
2,[2]
9,[9]


In [101]:
proportions('SAT2')
# La disposición a estudiar de nuevo algún grado universitario es muy superior a la disposición a ...
# ... volver a elegir los mismos estudios. 

1    0.615115
2    0.365612
9    0.019273
Name: SAT2, dtype: float64


In [102]:
# Distribución de la disposición a estudiar la misma titulación si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('SAT2','RAMA')
# La mayor disposición a estudiar de nuevo la misma titulación se da entre los graduados en ciencias de la salud. 

SAT2
RAMA SAT2          
1    1     0.630154
     2     0.352849
     9     0.016997
2    1     0.585884
     2     0.398272
     9     0.015844
3    1     0.593905
     2     0.385480
     9     0.020615
4    1     0.592394
     2     0.389113
     9     0.018494
5    1     0.725045
     2     0.255125
     9     0.019831

In [103]:
# Distribución de la disposición a estudiar de nuevo la misma titulación según las motivaciones para ...
# ... estudiar un grado: 
dist_condicionada('SAT2','EST_B8')
# Quienes estudiaron principalmente para ampliar conocimientos por satifacción personal se muestran más ...
# ... dispuestos a estudiar de nuevo la misma titulación. 

SAT2
EST_B8 SAT2          
1      1     0.601318
       2     0.380222
       9     0.018460
2      1     0.684283
       2     0.299018
       9     0.016699
3      1     0.609448
       2     0.363866
       9     0.026686
9      1     0.530435
       2     0.347826
       9     0.121739

#### Variables sobre otra formación realizada: variables  'EST_B11_1' a 'EST_B11_7'
Nota: Debe tenerse en cuenta que se pregunta sobre otros estudios completados, no sobre los iniciados, y que para los graduados recientes se pueden estar subestimando las titulaciones posteriores por no haber habido tiempo suficiente para terminarlas. 

#### 'EST_B11_1'     
¿Ha completado **otros estudios de grado**, diplomatura, licenciatura o equivalente?  1=Sí, 2=No, 9=NS/NC.

In [104]:
df_grad['EST_B11_1'].unique()

array([1, 2])

In [105]:
proportions('EST_B11_1')

2    0.765505
1    0.234495
Name: EST_B11_1, dtype: float64


In [106]:
# Distribución de uno o varios grados según rama de conocimiento de la titulación:
dist_condicionada('EST_B11_1','RAMA')
# Los graduados en ciencias sociales y jurídicas, y en ingeniería y arquitectura son los que ...
# ... tienen más de un grado con mayor frecuencia. 
# Los graduados en ciencias de la salud son los que con menos frecuencia tienen más de un grado, a pesar de que ...
# ... también son los que con más frecuencia manifiestan su disposición a estudiar si tuvieran que hacerlo ...
# ... de nuevo (el mismo u otro grado) (variables 'SAT1' y 'SAT2'). 

EST_B11_1
RAMA EST_B11_1           
1    1           0.181303
     2           0.818697
2    1           0.149802
     2           0.850198
3    1           0.280405
     2           0.719595
4    1           0.260552
     2           0.739448
5    1           0.137255
     2           0.862745

In [107]:
# Distribución de la disposición a estudiar de nuevo según se tengan uno o varios grados: 
dist_condicionada('SAT1','EST_B11_1')
# La disposición a estudiar si tuvieran que hacerlo de nuevo es algo mayor entre quienes tienen más de un grado. 

SAT1
EST_B11_1 SAT1          
1         1     0.884398
          2     0.097683
          9     0.017920
2         1     0.849808
          2     0.135210
          9     0.014982

In [108]:
# Distribución de uno o varios grados según el sexo:
dist_condicionada('EST_B11_1','SEXO')
# No hay diferencias significativas 

EST_B11_1
SEXO EST_B11_1           
1    1           0.238838
     2           0.761162
2    1           0.231225
     2           0.768775

In [109]:
# Distribución de uno o varios grados según las motivaciones para estudiar: 
dist_condicionada('EST_B11_1','EST_B8')
# Tener más de un grado es más frecuente entre quienes no declaran sus motivaciones ...
# ... para estudiar (NS/NC en 'EST_B11_1').
# Tener más de un grado es más frecuente entre quienes estudiaron principalmente para ampliar conocimientos ...
# ... por satifacción personal que entre quienes estudiaron principalmente por las espectativas profesionales. 

EST_B11_1
EST_B8 EST_B11_1           
1      1           0.214084
       2           0.785916
2      1           0.318271
       2           0.681729
3      1           0.243419
       2           0.756581
9      1           0.513043
       2           0.486957

#### 'EST_B11_2'     
¿Ha completado otros estudios de **máster universitario**?  1=Sí, 2=No, 9=NS/NC.

In [110]:
df_grad['EST_B11_2'].unique()

array([2, 1])

In [111]:
proportions('EST_B11_2')
# La proporcion de graduados con un máster universitario ronda el 50 %. 
# Es menos frecuente que tener una segunda titulación de grado. 

2    0.52526
1    0.47474
Name: EST_B11_2, dtype: float64


In [112]:
# Distribución de estudios de máster universitario según uno o varios grados:
dist_condicionada('EST_B11_2','EST_B11_1')
# Es más frecuente tener un máster universitario entre quienes tienen solo un título de grado. 

EST_B11_2
EST_B11_1 EST_B11_2           
1         1           0.389518
          2           0.610482
2         1           0.500846
          2           0.499154

In [113]:
# Distribución de estudios de máster universitario según las motivaciones para estudiar: 
dist_condicionada('EST_B11_2','EST_B8')
# Diferencias pequeñas entre los que no responden NS/NC.

EST_B11_2
EST_B8 EST_B11_2           
1      1           0.475098
       2           0.524902
2      1           0.478585
       2           0.521415
3      1           0.453300
       2           0.546700
9      1           0.747826
       2           0.252174

In [114]:
# Distribución de estudios de máster universitario según rama de conocimiento de la titulación:
dist_condicionada('EST_B11_2','RAMA')
# Los graduados en ciencias y en artes y humanidades son quienes con mayor frecuencia tienen estudios de máster. 
# La menor frecuencia se da entre los graduados en ingeniería y arquitectura.

EST_B11_2
RAMA EST_B11_2           
1    1           0.659742
     2           0.340258
2    1           0.708318
     2           0.291682
3    1           0.413748
     2           0.586252
4    1           0.402983
     2           0.597017
5    1           0.503565
     2           0.496435

In [115]:
# Distribución de estudios de máster universitario según el sexo:
dist_condicionada('EST_B11_2','SEXO')
# Entre los graduados, las mujeres son ligeramente más propensas que los hombres a realizar estudios de máster. 

EST_B11_2
SEXO EST_B11_2           
1    1           0.473851
     2           0.526149
2    1           0.475410
     2           0.524590

In [116]:
# Distribución del sexo según se tengan o no estudios de máster universitario:
dist_condicionada('EST_B11_2','SEXO',rev=True)
# Las mujeres también son mayoría entre los graduados. En consecuencia, el número de mujeres con estudios de ...
# ... máster es algo superior al de hombres. 

SEXO
EST_B11_2 SEXO          
1         1     0.428724
          2     0.571276
2         1     0.430256
          2     0.569744

#### 'EST_B11_3'     
¿Ha completado otros estudios de **doctorado universitario**?  1=Sí, 2=No, 9=NS/NC.

In [117]:
df_grad['EST_B11_3'].unique()

array([2, 1])

In [118]:
proportions('EST_B11_3')
# Tener un doctorado universitario es mucho menos frecuente que tener un máster universitario. 

2    0.984645
1    0.015355
Name: EST_B11_3, dtype: float64


In [119]:
# Distribución de estudios de doctorado según el sexo: 
dist_condicionada('EST_B11_3','SEXO')
# Tener un doctorado es más frecuente entre los hombres graduados que entre las mujeres graduadas. 
# La diferencia es suficiente para contrarestar que hay más mujeres entre los graduados, haciendo que ...
# ... la proporción de doctores sea mayor que la de doctoras. En efecto, (celda siguiente)

EST_B11_3
SEXO EST_B11_3           
1    1           0.018978
     2           0.981022
2    1           0.012627
     2           0.987373

In [120]:
dist_condicionada('EST_B11_3','SEXO',rev=True)
# Entre los doctores, la proporción hombres es superior a la de mujeres. 

SEXO
EST_B11_3 SEXO          
1         1     0.530864
          2     0.469136
2         1     0.427948
          2     0.572052

In [121]:
# Distribucion de estudios de doctorado según la motivación para estudiar: 
dist_condicionada('EST_B11_3','EST_B8')
# Los que no saben o no declaran cuáles son sus motivaciones para estudiar tienen mayor probabilidad de ...
# ... haber terminado un doctorado (debe tenerse en cuenta que son una proporción minúscula). 
# ... Le siguen a distancia lo que estudian para adquirir conocimientos.

EST_B11_3
EST_B8 EST_B11_3           
1      1           0.013180
       2           0.986820
2      1           0.027308
       2           0.972692
3      1           0.009376
       2           0.990624
9      1           0.078261
       2           0.921739

In [122]:
# Distribucion de las motivaciones para estudiar según se tenga o no el título de doctor: 
dist_condicionada('EST_B11_3','EST_B8',rev=True)
# La motivación para estudiar más frecuente de quienes terminan un doctorado son las expectativas laborales. 
# Sin embargo, esta motivación es menos frecuente entre los doctores que entre los no doctores. Ocurre lo ...
# ... contrario con la motivación de adquirir conocimientos por satisfacción personal. 

EST_B8
EST_B11_3 EST_B8          
1         1       0.641975
          2       0.286008
          3       0.053498
          9       0.018519
2         1       0.749591
          2       0.158864
          3       0.088144
          9       0.003401

In [123]:
# Distribucion de estudios de doctorado según las ramas de conocimiento: 
dist_condicionada('EST_B11_3','RAMA')
# La frecuencia más alta de doctores se da entre los graduados en ciencias. También son los que ...
# ... terminan un máster universitario con mayor frecuencia (véase la celda siguiente). 
# Los graduados en ciencias sociales y jurídicas con los menos propensos a terminar un doctorado. También ...
# ... son los segundos menos propensos a estudiar un máster universitario. 
# Para interpretar estos resultados debe tenerse en cuenta la mayor duración de los estudios de doctorado ...
# ... (incluyendo el tiempo dedicado a la tesis). Hace más probable que haya encuestados que cursan estos ...
# ... estudios sin haberlos concluido. 
# Con los datos disponibles no podemos inferir cuántos de los estudios de máster son habilitantes para el ...
# ... doctorado, que podrían considerarse estudios de doctorado no terminados. 

EST_B11_3
RAMA EST_B11_3           
1    1           0.020145
     2           0.979855
2    1           0.060857
     2           0.939143
3    1           0.006067
     2           0.993933
4    1           0.014019
     2           0.985981
5    1           0.015820
     2           0.984180

In [124]:
# Distribucion de estudios de máster universitario según las ramas de conocimiento: 
dist_condicionada('EST_B11_2','RAMA')
# Para facilitar comparación con resultados de la celda anterior. 

EST_B11_2
RAMA EST_B11_2           
1    1           0.659742
     2           0.340258
2    1           0.708318
     2           0.291682
3    1           0.413748
     2           0.586252
4    1           0.402983
     2           0.597017
5    1           0.503565
     2           0.496435

#### 'EST_B11_4'     
¿Ha completado otros **estudios artísticos superiores** (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)?  1=Sí, 2=No, 9=NS/NC.

In [125]:
df_grad['EST_B11_4'].unique()

array([2, 1])

In [126]:
proportions('EST_B11_4')
# Los estudios artísticos superiores son menos frecuentes que los de doctorado. 

2    0.992007
1    0.007993
Name: EST_B11_4, dtype: float64


In [127]:
# Distribución de estudios artísticos superiores según las ramas de conocimiento: 
dist_condicionada('EST_B11_4','RAMA')
# La frecuencia más alta corresponde a los graduados en artes y humanidades (la única superior al 1 %). 

EST_B11_4
RAMA EST_B11_4           
1    1           0.026755
     2           0.973245
2    1           0.007202
     2           0.992798
3    1           0.006826
     2           0.993174
4    1           0.004773
     2           0.995227
5    1           0.003788
     2           0.996212

In [128]:
# Distribución de las ramas de conocimiento según se tengan o no estudios artísticos superiores: 
dist_condicionada('EST_B11_4','RAMA',rev=True)
# Sin embargo, la rama de conocimiento más frecuente entre quienes tienen estudios artísticos superiores ...
# ... son las ciencias sociales y jurídicas. Esto se debe a que representan casi la mitad de los ...
# ... graduados (véase la celda siguiente).

RAMA
EST_B11_4 RAMA          
1         1     0.335968
          2     0.079051
          3     0.391304
          4     0.126482
          5     0.067194
2         1     0.098478
          2     0.087808
          3     0.458787
          4     0.212529
          5     0.142398

In [129]:
proportions ('RAMA')

3    0.458248
4    0.211842
5    0.141796
1    0.100376
2    0.087738
Name: RAMA, dtype: float64


In [130]:
# Distribución de estudios artísticos superiores según las motivaciones para estudiar: 
dist_condicionada('EST_B11_4','EST_B8')
# Quienes estudian por satisfacción personal y por 'otras' razones son quienes tienen estudios artísticos ...
# ... superiores con mayor frecuencia. La menor frecuencia corresponde a quienes estudian para mejorar sus ...
# ... expectativas profesionales. 

EST_B11_4
EST_B8 EST_B11_4           
1      1           0.006590
       2           0.993410
2      1           0.012574
       2           0.987426
3      1           0.011540
       2           0.988460
9      1           0.008696
       2           0.991304

In [131]:
# Distribución de las motivaciones para estudiar según se tengan o no estudios artísticos superiores: 
dist_condicionada('EST_B11_4','EST_B8',rev=True)
# La motivación mayoritaria para estudiar de quienes tienen estudios artísticos superiores son las ...
# ... expectativas profesionales, aunque en proporción inferior a la general. 

EST_B8
EST_B11_4 EST_B8          
1         1       0.616601
          2       0.252964
          3       0.126482
          9       0.003953
2         1       0.748997
          2       0.160074
          3       0.087299
          9       0.003631

In [132]:
# Distribución de estudios artísticos superiores según el sexo: 
dist_condicionada('EST_B11_4','SEXO')
# La proporcion de estudios artísticos superiores entre los graduados es mayor para las mujeres 
# ... que para los hombres. En consecuencia, (celda siguiente)

EST_B11_4
SEXO EST_B11_4           
1    1           0.006473
     2           0.993527
2    1           0.009138
     2           0.990862

In [133]:
# Distribución del sexo según se tengan o no estudios artísticos superiores: 
dist_condicionada('EST_B11_4','SEXO',rev=True)
# El número de mujeres casi dobla el de hombres entre quienes tienen estudios artísticos superiores. 

SEXO
EST_B11_4 SEXO          
1         1     0.347826
          2     0.652174
2         1     0.430187
          2     0.569813

#### 'EST_B11_5'     
¿Ha completado algún **ciclo formativo de grado superior** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado superior**?  1=Sí, 2=No, 9=NS/NC.

In [134]:
df_grad['EST_B11_5'].unique()

array([2, 1])

In [135]:
proportions('EST_B11_5')
# La proporción de graduados con ciclos formativos o enseñanzas deportivas de grado superior (13 %) es menor ...
# ... que la de un segundo título de grado (23 %) y muy superior a la de estudios de doctorado (1.5 %). 

2    0.871758
1    0.128242
Name: EST_B11_5, dtype: float64


In [136]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según el sexo: 
dist_condicionada('EST_B11_5','SEXO')
# La proporcion de ciclos formativos y enseñanzas deportivas de grado superior entre los graduados es mayor ...
# ... para las mujeres que para los hombres. En consecuencia, y puesto que las mujeres son mayoria entre ...
# ... los graduados, (celda siguiente)

EST_B11_5
SEXO EST_B11_5           
1    1           0.122692
     2           0.877308
2    1           0.132421
     2           0.867579

In [137]:
# Distribución del sexo según se tengan o no ciclos formativos y enseñanzas deportivas de grado superior: 
dist_condicionada('EST_B11_5','SEXO',rev=True)
# El número de mujeres es superior al de hombres entre quienes tienen ciclos formativos o enseñanzas ...
# ... deportivas de grado superior.

SEXO
EST_B11_5 SEXO          
1         1     0.410939
          2     0.589061
2         1     0.432263
          2     0.567737

In [138]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según la rama de conocimiento: 
dist_condicionada('EST_B11_5','RAMA')
# Los graduados en ciencias de la salud y en ciencias sociales y jurídicas son los que con mayor frecuencia ...
# ... tienen ciclos formativos y enseñanzas deportivas de grado superior.

EST_B11_5
RAMA EST_B11_5           
1    1           0.088763
     2           0.911237
2    1           0.076341
     2           0.923659
3    1           0.144650
     2           0.855350
4    1           0.114243
     2           0.885757
5    1           0.156194
     2           0.843806

In [139]:
# Distribución de las ramas de conocimiento según se tengan o no ciclos formativos y enseñanzas deportivas ...
# ... de grado superior: 
dist_condicionada('EST_B11_5','RAMA',rev=True)
# Más de la mitad de los graduados que tienen ciclos formativos o enseñanzas deportivas de grado superior ...
# ... lo son en ciencias sociales o jurídicas. 

RAMA
EST_B11_5 RAMA          
1         1     0.069475
          2     0.052230
          3     0.516876
          4     0.188716
          5     0.172703
2         1     0.104922
          2     0.092962
          3     0.449623
          4     0.215244
          5     0.137250

In [140]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según doctorado: 
dist_condicionada('EST_B11_5','EST_B11_3')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior es más de tres ..
# ... veces superior entre los no doctores que entre los doctores. 

EST_B11_5
EST_B11_3 EST_B11_5           
1         1           0.039095
          2           0.960905
2         1           0.129633
          2           0.870367

In [141]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según estudios de máster: 
dist_condicionada('EST_B11_5','EST_B11_2')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior entre los ...
# ... no doctores es el doble que entre los doctores. 

EST_B11_5
EST_B11_2 EST_B11_5           
1         1           0.082790
          2           0.917210
2         1           0.169323
          2           0.830677

In [142]:
# Distribución de las motivaciones para estudiar según se tengan o no ciclos formativos y enseñanzas ...
# ... deportivas de grado superior: 
dist_condicionada('EST_B11_5','EST_B8',rev=True)
# No hay diferencias notables. 

EST_B8
EST_B11_5 EST_B8          
1         1       0.728505
          2       0.166051
          3       0.102488
          9       0.002956
2         1       0.750797
          2       0.160046
          3       0.085423
          9       0.003733

In [143]:
# Distribución de los ciclos formativos y enseñanzas deportivas de grado superior según motivacion para estudiar: 
dist_condicionada('EST_B11_5','EST_B8')
# De nuevo, sin diferencias notables. 

EST_B11_5
EST_B8 EST_B11_5           
1      1           0.124910
       2           0.875090
2      1           0.132417
       2           0.867583
3      1           0.150018
       2           0.849982
9      1           0.104348
       2           0.895652

In [144]:
# Distribución de los ciclos formativos y enseñanzas deportivas de grado superior según la edad: 
dist_condicionada('EST_B11_5','EDAD')
# El grupo de gradudados de edad de 30 a 34 es el que con mayor frecuencia tiene un título de ciclos formativos o ...
# ... enseñanzas deportivas de grado superior. 
# La diferencia de frecuencia es suficiente para contrarestar que los graduados de menos de 30 años son muchos ...
# ... más. En efecto, (celda siguiente)

EST_B11_5
EDAD EST_B11_5           
1    1           0.047495
     2           0.952505
2    1           0.217450
     2           0.782550
3    1           0.195814
     2           0.804186

In [145]:
dist_condicionada('EST_B11_5','EDAD',rev=True)
# La edad más frecuente entre los graduados que tienen un título de ciclos formativos o enseñanzas deportivas ...
# ... de grado superior es el grupo de 30 a 34 años. 

EDAD
EST_B11_5 EDAD          
1         1     0.183789
          2     0.472777
          3     0.343434
2         1     0.542222
          2     0.250290
          3     0.207488

#### 'EST_B11_6'     
¿Ha completado algún **ciclo formativo de grado medio** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado medio**?  1=Sí, 2=No, 9=NS/NC.

In [160]:
df_grad['EST_B11_6'].unique()

array([2, 1])

In [161]:
proportions('EST_B11_6')
# Menos frecuentes que los mismos estudios de grado superior, más frecuentes que estudios artisticos superiores

2    0.966668
1    0.033332
Name: EST_B11_6, dtype: float64


In [168]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según esos estudios en su ...
# ... grado superior: 
dist_condicionada('EST_B11_6','EST_B11_5')
# Solo el 11 % de los graduados que han completado ciclos de grado superior proceden de esos estudios en su ...
# ... grado medio. Es decir, casi el 90 % de los graduados universitarios que terminan ciclos formativos ...
# ... o educación deportiva de grado superior acceden a esos estudios sin haber cursado los de grado medio. 

EST_B11_6
EST_B11_5 EST_B11_6           
1         1           0.107908
          2           0.892092
2         1           0.022362
          2           0.977638

In [170]:
# Distribución de los ciclos formativos y educación deportiva de grado superior según esos estudios en su ...
# ... grado medio:
dist_condicionada('EST_B11_6','EST_B11_5',rev=True)
# El 42 % los graduados con ciclos formativos o educación deportiva de grado medio completa esos estudios en ...
# ... grado superior. 

EST_B11_5
EST_B11_6 EST_B11_5           
1         1           0.415166
          2           0.584834
2         1           0.118349
          2           0.881651

In [171]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según el sexo: 
dist_condicionada('EST_B11_6','SEXO')
# Las distribuciones condicionadas por el sexo son casi idénticas.  

EST_B11_6
SEXO EST_B11_6           
1    1           0.033689
     2           0.966311
2    1           0.033064
     2           0.966936

#### 'EST_B11_7'     
No ha completado **ninguno de los estudios anteriores**: 1=Sí, 2=No, 9=NS/NC.

In [155]:
df_grad['EST_B11_7'].unique()

array([2, 1])

In [156]:
df_grad[['EST_B11_1','EST_B11_2','EST_B11_3','EST_B11_4','EST_B11_5','EST_B11_6','EST_B11_7']].groupby(
        ['EST_B11_7']).agg(['unique'])
# Comprobación: solo toma valor 1 cuando todas las variables de otros estudios anteriores toman valor 2. 

,EST_B11_1,EST_B11_2,EST_B11_3,EST_B11_4,EST_B11_5,EST_B11_6
,unique,unique,unique,unique,unique,unique
EST_B11_7,,,,,,
1,[2],[2],[2],[2],[2],[2]
2,"[1, 2]","[2, 1]","[2, 1]","[2, 1]","[2, 1]","[2, 1]"


#### 'EST_B12'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1')  
**Número de otros estudios de grado**, diplomatura, licenciatura o equivalente (aparte de aquellos a los que se refiere la encuesta): 1=Uno, 2=Dos, 3=Tres o más.

In [172]:
df_grad['EST_B12'].unique()

array(['2', '1', ' ', '3'], dtype=object)

In [173]:
df_grad[['EST_B11_1','EST_B12']].groupby('EST_B11_1').agg(['unique'])

,EST_B12
,unique
EST_B11_1,
1,"[2, 1, 3]"
2,[ ]


In [175]:
# Se imputa valor 0 a quienes declararon no haber realizado otros estudios de este tipo: 
df_grad['EST_B12'].replace({' ':'0'},inplace=True)
df_grad[['EST_B11_1','EST_B12']].groupby('EST_B11_1').agg(['unique'])

,EST_B12
,unique
EST_B11_1,
1,"[2, 1, 3]"
2,[0]


In [176]:
proportions('EST_B12')

0    0.765505
1    0.204133
2    0.025181
3    0.005182
Name: EST_B12, dtype: float64


In [178]:
# Distribución de doctores y no doctores según el número de títulos de grado:  
dist_condicionada('EST_B12','EST_B11_3',rev=True)
# Terminar los estudios de doctorado es más frecuente cuanto mayor es el número de títulos de grado. 

EST_B11_3
EST_B12 EST_B11_3           
0       1           0.013414
        2           0.986586
1       1           0.019192
        2           0.980808
2       1           0.036386
        2           0.963614
3       1           0.048780
        2           0.951220

In [186]:
# Distribución de número de grados universitarios según se tenga o no un título de doctor: 
dist_condicionada('EST_B12','EST_B11_3')
# Los doctores tienen más de un grado universitario con más frecuencia que los no doctores. En concreto, ...
# ... ser doctor multiplica por más de 2 la probabilidad de tener 3 grados y por 3 la de tener 4 o más títulos ...
# ... de grado (incluyendo aquel al que se refiere la encuesta). 

EST_B12
EST_B11_3 EST_B12          
1         0        0.668724
          1        0.255144
          2        0.059671
          3        0.016461
2         0        0.767014
          1        0.203337
          2        0.024643
          3        0.005006

In [181]:
# Distribución de títulos de máster universitario según el número de títulos de grado:  
dist_condicionada('EST_B12','EST_B11_2',rev=True)
# Los titulados en un único grado universitario son los que con más frecuencia terminan estudios de máster ...
# ... universitario. Les siguen quienes tienen 4 o más títulos de grado. 

EST_B11_2
EST_B12 EST_B11_2           
0       1           0.500846
        2           0.499154
1       1           0.391890
        2           0.608110
2       1           0.363864
        2           0.636136
3       1           0.420732
        2           0.579268

In [185]:
# Distribución de número de grados universitarios según se tenga o no un título de máster universitario: 
dist_condicionada('EST_B12','EST_B11_2')
# Quienes terminan un máster universitario tienen solo un título de grado con más frecuencia que quienes ...
# ... no tienen un título de máster. 

EST_B12
EST_B11_2 EST_B12          
1         0        0.807600
          1        0.168508
          2        0.019300
          3        0.004592
2         0        0.727459
          1        0.236331
          2        0.030496
          3        0.005714

In [183]:
# Distribución de número de grados universitarios según el sexo: 
dist_condicionada('EST_B12','SEXO')
# Las diferencias entre sexos son pequeñas. 

EST_B12
SEXO EST_B12          
1    0        0.761162
     1        0.208533
     2        0.024347
     3        0.005958
2    0        0.768775
     1        0.200820
     2        0.025809
     3        0.004597

In [189]:
# Distribución de número de grados universitarios según la edad: 
dist_condicionada('EST_B12','EDAD')
# Como es de esperar, a medida que aumenta la edad aumenta también la frecuencia de titulados en un número ...
# ... mayor de grados. 

EST_B12
EDAD EST_B12          
1    0        0.881072
     1        0.111988
     2        0.005857
     3        0.001082
2    0        0.747082
     1        0.225382
     2        0.024816
     3        0.002720
3    0        0.533361
     1        0.381093
     2        0.068268
     3        0.017278

#### 'EST_B13_AMB1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1')  
**Ambito de estudio del segundo grado**, diplomatura, licenciatura o equivalente en el que está titulado: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.

In [190]:
df_grad['EST_B13_AMB1'].unique()

array(['011 ', '    ', '0113', '0112', '092 ', '0313', '9999', '023 ',
       '1014', '0413', '091 ', '1015', '073 ', '031 ', '041 ', '042 ',
       '071 ', '053 ', '032 ', '051 ', '052 ', '0913', '021 ', '022 ',
       '081 ', '0211', '054 ', '0912', '0311', '082 ', '061 ', '109 ',
       '084 ', '072 '], dtype=object)

In [199]:
df_grad[['EST_B11_1','EST_B13_AMB1']].groupby('EST_B11_1').agg(['unique'])

,EST_B13_AMB1
,unique
EST_B11_1,
1,"[011 , 0113, 0112, 092 , 0313, 9999, 023 , 1014, 0413, 091 , 1015, 073 , 031 , 041 , 042 , 071 , 053 , 032 , 051 , 052 , 0913, 021 , 022 , 081 , 0211, 054 , 0912, 0311, 082 , 061 , 109 , 084 , 072 ]"
2,[ ]


In [200]:
# El ámbito de estudio es menos manejabel que la rama de conocimiento, que es un criterio más sintético. 
# Para identificar la rama que corresponde a cada ámbito: 
df_grad[['RAMA','AMBITO']].groupby('AMBITO').agg(['value_counts'])

RAMA
            value_counts
AMBITO RAMA             
11     3            1061
       1               1
21     1             887
       4              19
22     1             916
23     1            1149
       3               6
31     3             547
       1             160
32     3             769
       1              15
41     3             851
42     3            1720
51     2             882
       5              61
52     2             489
53     2             761
       3              63
       1              38
       4              26
54     2             387
       3              34
       4              24
61     4            1132
71     4            2605
       2               4
72     4             321
       2             203
       5              36
73     4            1541
       1               1
81     4             545
82     4             303
84     5             422
91     5            1254
       2              51
       4              50
       3               1
92     3             810
109    4             135
       3              85
112    3            1365
113    3            1907
211    3             515
       1              10
       4               4
311    3             763
313    5             629
       3             388
413    3            2515
912    5             767
913    5            1235
1014   3             519
       5              84
1015   3             585

In [204]:
# Cuando hay varias ramas asociadas a un ámbito, elegimos la correspondencia más frecuente. 
